In [1]:
# ----------------------------------------------------------
# DRIVER DROWSINESS DETECTION - HIGH ACCURACY VERSION
# ----------------------------------------------------------

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# ✅ STEP 1: PATHS
train_dir = r'C:\Users\Dibyajyoti\OneDrive\Documents\AI ASSIGNMENTS\MACHINE LEARNING AND DEEPLEARNING ASSIGNMENTS\Drowsiness_Project\dataset\Driver Drowsiness and Distraction Detection Dataset-20251026T115350Z-1-001\train'
test_dir  = r'C:\Users\Dibyajyoti\OneDrive\Documents\AI ASSIGNMENTS\MACHINE LEARNING AND DEEPLEARNING ASSIGNMENTS\Drowsiness_Project\dataset\Driver Drowsiness and Distraction Detection Dataset-20251026T115350Z-1-001\test'

print("✅ Using training path:", train_dir)
print("✅ Using testing path:", test_dir)

# ----------------------------------------------------------
# STEP 2: DATA GENERATORS WITH AUGMENTATION
# ----------------------------------------------------------
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

# ----------------------------------------------------------
# STEP 3: MODEL (TRANSFER LEARNING + FINE TUNING)
# ----------------------------------------------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

# ✅ Unfreeze top layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

# ----------------------------------------------------------
# STEP 4: COMPILE
# ----------------------------------------------------------
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# ----------------------------------------------------------
# STEP 5: CALLBACKS (stabilize and boost training)
# ----------------------------------------------------------
early_stop = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1)

# ----------------------------------------------------------
# STEP 6: TRAIN LONGER
# ----------------------------------------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=25,
    callbacks=[early_stop, lr_reduce]
)

# ----------------------------------------------------------
# STEP 7: SAVE MODEL
# ----------------------------------------------------------
os.makedirs('models', exist_ok=True)
model_path = os.path.join('models', 'drowsiness_model_v2.h5')
model.save(model_path)
print("✅ Model saved at:", model_path)

# ----------------------------------------------------------
# STEP 8: FINAL EVALUATION
# ----------------------------------------------------------
loss, acc = model.evaluate(test_data)
print(f"\n✅ Final Test Accuracy: {acc*100:.2f}%")


✅ Using training path: C:\Users\Dibyajyoti\OneDrive\Documents\AI ASSIGNMENTS\MACHINE LEARNING AND DEEPLEARNING ASSIGNMENTS\Drowsiness_Project\dataset\Driver Drowsiness and Distraction Detection Dataset-20251026T115350Z-1-001\train
✅ Using testing path: C:\Users\Dibyajyoti\OneDrive\Documents\AI ASSIGNMENTS\MACHINE LEARNING AND DEEPLEARNING ASSIGNMENTS\Drowsiness_Project\dataset\Driver Drowsiness and Distraction Detection Dataset-20251026T115350Z-1-001\test
Found 2218 images belonging to 4 classes.
Found 464 images belonging to 4 classes.


c:\Users\Dibyajyoti\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 956s 13s/step - accuracy: 0.7588 - loss: 0.6205 - val_accuracy: 0.7608 - val_loss: 0.5434 - learning_rate: 1.0000e-04
Epoch 2/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 68s 965ms/step - accuracy: 0.9247 - loss: 0.1948 - val_accuracy: 0.7759 - val_loss: 0.5406 - learning_rate: 1.0000e-04
Epoch 3/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 65s 930ms/step - accuracy: 0.9585 - loss: 0.1059 - val_accuracy: 0.8233 - val_loss: 0.4847 - learning_rate: 1.0000e-04
Epoch 4/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9757 - loss: 0.0626 - val_accuracy: 0.8103 - val_loss: 0.6445 - learning_rate: 1.0000e-04
Epoch 5/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9861 - loss: 0.0543
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
70/70 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.9820 - loss: 0.0634 - val_accuracy: 0.8125 - val_loss: 0.6560 - learning_rate: 1.0000e-04
Epoch 6/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9883 - los

✅ Model saved at: models\drowsiness_model_v2.h5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - accuracy: 0.9030 - loss: 0.2652

✅ Final Test Accuracy: 90.30%
